In [1]:
!pip install xgboost
!pip install seaborn
!pip install pandas scikit-learn

In [2]:
import pandas as pd
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from collections import defaultdict
from pathlib import Path
import os
import itertools 
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
seed =1380
np.random.seed(seed)
random.seed(seed) 
os.environ['PYTHONHASHSEED']=str(seed)

## **COBRE 127 subjects**

In [4]:
cobre127 = pd.read_csv('cobre_127.csv')

In [5]:
cobre127

,"('Precentral_R', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_R')","('Frontal_Sup_R', 'Precentral_L')","('Frontal_Sup_R', 'Precentral_R')","('Frontal_Sup_R', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Precentral_L')","('Frontal_Sup_Orb_L', 'Precentral_R')","('Frontal_Sup_Orb_L', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Frontal_Sup_R')",...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",Subjectid,Dx
0,0.590307,0.215914,0.304414,0.116425,0.165778,0.255798,0.031173,-0.113507,0.038880,0.170122,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,A00000368,1
1,0.600053,0.450454,0.407950,0.554324,0.457576,0.747958,0.101570,-0.024283,0.218750,0.244049,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,A00000456,1
2,0.640427,0.425068,0.470462,0.313035,0.265838,0.615625,-0.134986,-0.242692,-0.040470,0.093006,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,A00000838,1
3,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,A00000909,1
4,0.664985,0.299166,0.073119,0.407197,0.240695,0.669216,0.168402,0.111856,0.097451,0.082961,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,A00001181,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0.787778,0.418824,0.420073,0.536880,0.531874,0.776564,0.222002,0.235727,-0.130599,0.042551,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,A00037649,1
123,0.837095,0.699350,0.567253,0.628090,0.484317,0.816644,0.515332,0.440825,0.597178,0.574797,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,A00037665,0
124,0.626127,0.367332,0.178448,0.252641,0.384334,0.654329,0.114585,0.223127,-0.013945,0.164300,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,A00037854,1
125,0.457744,0.486875,0.204815,0.321213,0.284081,0.572457,-0.062898,-0.092247,-0.064799,-0.135443,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,A00038441,1


In [6]:
cobre = cobre127.iloc[:, :-2].values
targetcobre = cobre127['Dx'].values

In [7]:
targetcobre

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1])

## **1. Nested cross-validation for COBRE (127 subjects)**

### **1.1 Logistic Regression COBRE**

In [20]:
param_grid = [
    {'C': [0.01, 0.1, 1, 3, 10], 'solver': ['liblinear'], 'penalty': ['l1','l2'], 'random_state': [seed]}
]
model_to_tune = LogisticRegression()

search = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, n_jobs=-1
)
search.fit(cobre, targetcobre)

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 3, 10], 'penalty': ['l1', 'l2'],
                          'random_state': [1380], 'solver': ['liblinear']}])

In [25]:
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Fit the GridSearchCV model on the training data
model.fit(cobre, targetcobre)

# Compute the non-nested cross-validation scores for the tuned model
estimator = model.best_estimator_
test_score_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Calculate the difference between nested and non-nested cross-validation scores
score_difference = test_score_not_nested.mean() - test_score_nested.mean()
print(f"The mean F1 score using nested cross-validation is: "
      f"{test_score_nested.mean():.3f} ± {test_score_nested.std():.3f}")
print(f"The mean F1 score using non-nested cross-validation is: "
      f"{test_score_not_nested.mean():.3f} ± {test_score_not_nested.std():.3f}")
print(f"The difference between nested and non-nested F1 scores is: {score_difference:.3f}")

# Nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using nested cross-validation is: "
      f"{test_acc_nested.mean():.3f} ± {test_acc_nested.std():.3f}")

print(f"The mean ROC AUC using nested cross-validation is: "
      f"{test_auc_nested.mean():.3f} ± {test_auc_nested.std():.3f}")

# Non-nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using non-nested cross-validation is: "
      f"{test_acc_not_nested.mean():.3f} ± {test_acc_not_nested.std():.3f}")

print(f"The mean ROC AUC using non-nested cross-validation is: "
      f"{test_auc_not_nested.mean():.3f} ± {test_auc_not_nested.std():.3f}")


/opt/miniconda-latest/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda-latest/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda-latest/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda-latest/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda-latest/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/miniconda-latest/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinea

The mean F1 score using nested cross-validation is: 0.692 ± 0.109
The mean F1 score using non-nested cross-validation is: 0.727 ± 0.108
The difference between nested and non-nested F1 scores is: 0.035
The mean accuracy using nested cross-validation is: 0.692 ± 0.083
The mean ROC AUC using nested cross-validation is: 0.713 ± 0.099
The mean accuracy using non-nested cross-validation is: 0.740 ± 0.111
The mean ROC AUC using non-nested cross-validation is: 0.739 ± 0.096


### **1.2 Random Forest COBRE**

In [26]:
param_grid = [
    {'n_estimators': [10, 50, 100], 'max_depth':  [3,5,7,9], 'random_state': [seed]}
]

model_to_tune = RandomForestClassifier()

search = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, n_jobs=-1
)
search.fit(cobre, targetcobre)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid=[{'max_depth': [3, 5, 7, 9],
                          'n_estimators': [10, 50, 100],
                          'random_state': [1380]}])

In [27]:
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Fit the GridSearchCV model on the training data
model.fit(cobre, targetcobre)

# Compute the non-nested cross-validation scores for the tuned model
estimator = model.best_estimator_
test_score_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Calculate the difference between nested and non-nested cross-validation scores
score_difference = test_score_not_nested.mean() - test_score_nested.mean()
print(f"The mean F1 score using nested cross-validation is: "
      f"{test_score_nested.mean():.3f} ± {test_score_nested.std():.3f}")
print(f"The mean F1 score using non-nested cross-validation is: "
      f"{test_score_not_nested.mean():.3f} ± {test_score_not_nested.std():.3f}")
print(f"The difference between nested and non-nested F1 scores is: {score_difference:.3f}")

# Nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using nested cross-validation is: "
      f"{test_acc_nested.mean():.3f} ± {test_acc_nested.std():.3f}")

print(f"The mean ROC AUC using nested cross-validation is: "
      f"{test_auc_nested.mean():.3f} ± {test_auc_nested.std():.3f}")

# Non-nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using non-nested cross-validation is: "
      f"{test_acc_not_nested.mean():.3f} ± {test_acc_not_nested.std():.3f}")

print(f"The mean ROC AUC using non-nested cross-validation is: "
      f"{test_auc_not_nested.mean():.3f} ± {test_auc_not_nested.std():.3f}")

The mean F1 score using nested cross-validation is: 0.638 ± 0.121
The mean F1 score using non-nested cross-validation is: 0.701 ± 0.119
The difference between nested and non-nested F1 scores is: 0.063
The mean accuracy using nested cross-validation is: 0.669 ± 0.119
The mean ROC AUC using nested cross-validation is: 0.657 ± 0.116
The mean accuracy using non-nested cross-validation is: 0.724 ± 0.115
The mean ROC AUC using non-nested cross-validation is: 0.717 ± 0.118


### **1.3 SVM COBRE**

In [8]:
param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

In [9]:
param_grid = [
    {'C': [0.01, 0.1], 'kernel': ['linear', 'rbf'], 'random_state': [seed]}
]

model_to_tune = SVC()

search = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, n_jobs=-1
)
search.fit(cobre, targetcobre)

GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1], 'kernel': ['linear', 'rbf'],
                          'random_state': [1380]}])

In [10]:
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Fit the GridSearchCV model on the training data
model.fit(cobre, targetcobre)

# Compute the non-nested cross-validation scores for the tuned model
estimator = model.best_estimator_
test_score_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Calculate the difference between nested and non-nested cross-validation scores
score_difference = test_score_not_nested.mean() - test_score_nested.mean()
print(f"The mean F1 score using nested cross-validation is: "
      f"{test_score_nested.mean():.3f} ± {test_score_nested.std():.3f}")
print(f"The mean F1 score using non-nested cross-validation is: "
      f"{test_score_not_nested.mean():.3f} ± {test_score_not_nested.std():.3f}")
print(f"The difference between nested and non-nested F1 scores is: {score_difference:.3f}")

# Nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using nested cross-validation is: "
      f"{test_acc_nested.mean():.3f} ± {test_acc_nested.std():.3f}")

print(f"The mean ROC AUC using nested cross-validation is: "
      f"{test_auc_nested.mean():.3f} ± {test_auc_nested.std():.3f}")

# Non-nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using non-nested cross-validation is: "
      f"{test_acc_not_nested.mean():.3f} ± {test_acc_not_nested.std():.3f}")

print(f"The mean ROC AUC using non-nested cross-validation is: "
      f"{test_auc_not_nested.mean():.3f} ± {test_auc_not_nested.std():.3f}")

The mean F1 score using nested cross-validation is: 0.749 ± 0.079
The mean F1 score using non-nested cross-validation is: 0.741 ± 0.076
The difference between nested and non-nested F1 scores is: -0.007
The mean accuracy using nested cross-validation is: 0.764 ± 0.086
The mean ROC AUC using nested cross-validation is: 0.755 ± 0.069
The mean accuracy using non-nested cross-validation is: 0.756 ± 0.082
The mean ROC AUC using non-nested cross-validation is: 0.750 ± 0.065


In [11]:
print(f"The best parameters found are: {search.best_params_}")

The best parameters found are: {'C': 0.1, 'kernel': 'linear', 'random_state': 1380}


In [12]:
estimator

SVC(C=0.01, kernel='linear', random_state=1380)

### Important features for COBRE from SVC

In [14]:
import csv
feature_importances_svc = np.abs(estimator.coef_).mean(axis=0)
top_features_indices_svc = np.argsort(feature_importances_svc)[::-1][:50]

# Create a list to store the results
results = [("Feature", "Coefficient Value")]

# Iterate through the top features and append them to the results list
for index in top_features_indices_svc:
    feature_name = cobre127.columns[index]
    coefficient_value = estimator.coef_[0, index]
    results.append((feature_name, coefficient_value))

# Specify the CSV file path
csv_file_path = "important_features_COBRE.csv"

# Write the results to the CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(results)

print(f"Results saved to {csv_file_path}")

Results saved to important_features_COBRE.csv


In [15]:
print("Top 50 Important Features:")
for index in top_features_indices_svc:
    feature_name = cobre127.columns[index]
    coefficient_value = estimator.coef_[0, index]  
    print(f"{feature_name}: {coefficient_value}")

Top 50 Important Features:
('Cerebelum_8_R', 'Frontal_Sup_Orb_L'): -0.03855577115751565
('Cerebelum_10_L', 'Cerebelum_8_L'): 0.037467897704439236
('Cerebelum_8_L', 'Frontal_Sup_Orb_L'): -0.03473239687018249
('Occipital_Inf_R', 'Frontal_Sup_Medial_L'): -0.034595405631808886
('Cerebelum_7b_R', 'Frontal_Sup_Orb_L'): -0.03287065141842159
('Pallidum_R', 'Amygdala_R'): 0.03235001560503342
('Cerebelum_7b_R', 'Temporal_Pole_Mid_L'): 0.032046912182382294
('Vermis_4_5', 'Vermis_3'): 0.03182472575538329
('Cerebelum_10_L', 'Cerebelum_8_R'): 0.03165228644966857
('Cerebelum_9_L', 'Cerebelum_3_R'): 0.03131751653304107
('Cerebelum_8_R', 'Frontal_Mid_Orb_R'): -0.031211037653047478
('Cerebelum_4_5_L', 'Hippocampus_R'): 0.030877629787341165
('Temporal_Inf_L', 'Temporal_Pole_Mid_L'): 0.030803469991569552
('Cerebelum_10_L', 'Temporal_Pole_Sup_R'): 0.030580367686795114
('ParaHippocampal_R', 'Olfactory_R'): 0.030505015375680056
('Vermis_4_5', 'Hippocampus_R'): 0.030481166674126762
('Occipital_Inf_R', 'Fronta

### **1.4 XGBoost COBRE**

In [38]:
param_grid = [
                {'objective': ['binary:logistic'], 'eta': [0.1], 'max_depth': [10],
                 'scale_pos_weight': [1.0], 'alpha': [0], 'random_state': [seed]}
            ]

In [39]:
model_to_tune = XGBClassifier(use_label_encoder=False)

In [40]:
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_nested = cross_val_score(model, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Fit the GridSearchCV model on the training data
model.fit(cobre, targetcobre)

# Compute the non-nested cross-validation scores for the tuned model
estimator = model.best_estimator_
test_score_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
test_acc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring='accuracy')
test_auc_not_nested = cross_val_score(estimator, cobre, targetcobre, cv=outer_cv, n_jobs=-1, scoring=make_scorer(roc_auc_score))

# Calculate the difference between nested and non-nested cross-validation scores
score_difference = test_score_not_nested.mean() - test_score_nested.mean()
print(f"The mean F1 score using nested cross-validation is: "
      f"{test_score_nested.mean():.3f} ± {test_score_nested.std():.3f}")
print(f"The mean F1 score using non-nested cross-validation is: "
      f"{test_score_not_nested.mean():.3f} ± {test_score_not_nested.std():.3f}")
print(f"The difference between nested and non-nested F1 scores is: {score_difference:.3f}")

# Nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using nested cross-validation is: "
      f"{test_acc_nested.mean():.3f} ± {test_acc_nested.std():.3f}")

print(f"The mean ROC AUC using nested cross-validation is: "
      f"{test_auc_nested.mean():.3f} ± {test_auc_nested.std():.3f}")

# Non-nested Cross-Validation Accuracy and ROC AUC
print(f"The mean accuracy using non-nested cross-validation is: "
      f"{test_acc_not_nested.mean():.3f} ± {test_acc_not_nested.std():.3f}")

print(f"The mean ROC AUC using non-nested cross-validation is: "
      f"{test_auc_not_nested.mean():.3f} ± {test_auc_not_nested.std():.3f}")

The mean F1 score using nested cross-validation is: 0.729 ± 0.093
The mean F1 score using non-nested cross-validation is: 0.729 ± 0.093
The difference between nested and non-nested F1 scores is: 0.000
The mean accuracy using nested cross-validation is: 0.755 ± 0.083
The mean ROC AUC using nested cross-validation is: 0.739 ± 0.086
The mean accuracy using non-nested cross-validation is: 0.755 ± 0.083
The mean ROC AUC using non-nested cross-validation is: 0.739 ± 0.086


/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/miniconda-latest/lib